In [1]:
import tensorflow as tf

if tf.config.list_physical_devices('GPU'):
    device_name = tf.test.gpu_device_name()
else:
    device_name = 'CPU'
device_name = 'CPU' 
print(f'device name {device_name}')
import os
import sys

from math import cos,sin,tan,asin,acos,radians,sqrt,degrees,atan,atan2,copysign
import numpy as np

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import pickle
import scipy
from scipy.stats import norm
import random
import time
import timeit
import math
import localization as lx
import gzip

import util.npose_util as nu
import datetime
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial import cKDTree


import joblib
from sklearn.manifold import MDS
import argparse
from functools import partial
from itertools import starmap,repeat,permutations

from pymol import cmd, stored, selector

import GenerateEndpoints as ge
import HelixFit as hf
import FitTransform as ft

import seaborn as sns
import util.RotationMethods as rm

import copy




device name CPU


In [2]:
import Guided_Midpoint_Buttress as gmp

device name CPU


In [3]:
start_hel, gep= gmp.get_reference_input(batch=1000 )

In [4]:
out_ep = gmp.build_protein_on_guide(start_hel, gep, batch=200, 
                                 next_mp_dist = 11,mp_deviation_limit = 5)

max next indices 13
(1000, 8, 3)
Input Size:  100000
Passing Filters:  535
final pass filter 489
max next indices 29
(489, 8, 3)
Input Size:  48900
Passing Filters:  723
final pass filter 449
max next indices 47
(449, 8, 3)
Input Size:  44900
Passing Filters:  934
final pass filter 564
max next indices 66
(564, 8, 3)
Input Size:  56400
Passing Filters:  764
final pass filter 398
max next indices 89
(398, 8, 3)
Input Size:  39800
Passing Filters:  533
final pass filter 328
max next indices 99
(210, 8, 3)
Input Size:  21000
Passing Filters:  369
final pass filter 159


In [7]:
gmp.vp(out_ep[1],gep,max_out=100)

In [8]:
len(out_ep[1])

159

In [9]:
sumL = 0

for x in out_ep:
    sumL += len(x)
print(sumL)

277


In [10]:
import LoopEndpoints as le

In [18]:
def add_loops(end_points, neighbors=10, length_mod=1, dist_cut=6, phiQueryNum=10, randMult=10, maxPhi_cut=20,
              uniquePhi=True, verbose=False, maxReruns = 4):
    """Chains first_helix, first_loop, second helix and the iteratively loop_then_helix to add all loops to endpoints."""
    
    epIn = end_points.copy()
    size = int(len(epIn)/2) #number of helixes represented by helical endpoints
    
   
    bL, iL, epGuide = le.first_helix(epIn,length_mod=length_mod)
    if verbose:
        print(f'first helix: #{iL.shape[0]}')
    
    if iL.shape[0] == 0:
        if verbose:
            print('fail')
        return bL, iL, phiList, iL, iL, False
    
    bL, iL, epGuide, epTrue, phiList, loopFeature, xform_True = le.first_loop(bL, iL, epGuide, 
                                                                neighbors=neighbors, phiQueryNum=phiQueryNum,
                                                                           randMult=randMult, distCut=dist_cut)
    if verbose:
        print(f'first loop: #{iL.shape[0]}')
    if iL.shape[0] == 0:
        if verbose:
            print('fail')
        return bL, iL, phiList, iL, iL, False
    
    hnum=1
    bL, iL, epGuide, epTrue, phiList = le.second_helix(bL, iL, epGuide, epTrue, phiList, hnum, loopFeature, 
                                                   xform_True, length_mod=length_mod, distCut=dist_cut)
    if verbose:
        print(f'second helix: #{iL.shape[0]}')
    
    if iL.shape[0] == 0:
        if verbose:
            print('fail')
        return bL, iL, phiList, iL, iL, False
    
    repeatNumber = 0 
    hnum = 0
    
    bL_s = []
    iL_s = []
    epGuide_s = []
    epTrue_s = []
    phiList_s =  []
    for x in range(0,(size-1)):
        bL_s.append([])
        iL_s.append([])
        epGuide_s.append([])
        epTrue_s.append([])
        phiList_s.append([])
        
    
    while hnum<(size-2):
        hnum += 1
        print('hnum',hnum)
        bL, iL, epGuide, epTrue, phiList = le.next_loop_helix(bL, iL, epGuide, epTrue, phiList, hnum, 
                neighbors=neighbors, phiQueryNum=phiQueryNum, randMult=randMult, distCut=dist_cut,length_mod=length_mod)
       
        bL_s[hnum] = copy.deepcopy(bL)
        iL_s[hnum] = iL.copy()
        epGuide_s[hnum] = epGuide.copy()
        epTrue_s[hnum] = epTrue.copy()
        phiList_s[hnum] = phiList.copy()
        
        if iL.shape[0] == 0:
            
            hnum = hnum - 2
            
            bL = bL_s[hnum] 
            iL = iL_s[hnum] 
            epGuide = epGuide_s[hnum] 
            epTrue = epTrue_s[hnum] 
            phiList = phiList_s[hnum] 
            
            neighbors += 5
            maxPhi_cut += 100
            
            repeatNumber += 1
            print('rn:', repeatNumber)
                
            if repeatNumber == maxReruns:
                if verbose:
                    print('fail2')
                return bL, iL, phiList, iL, iL, False
            
        if iL.shape[0] > maxPhi_cut:
            indexer = le.random_reduce(bL,num_to_keep = maxPhi_cut)
            bL = bL[indexer]
            iL = iL[indexer] 
            epGuide = epGuide[indexer]
            epTrue =  epTrue[indexer]
            phiList = phiList[indexer]
            if verbose:
                print(f'{hnum+2} helix: reduced to #{maxPhi_cut}')
        if verbose:
            print(f'{hnum+2} helix: #{iL.shape[0]}')
            
         
    #only return proteins with a unique set of loops queried from different phi bins
    if uniquePhi:
        phiList, u_indices = np.unique(phiList,axis=0,return_index=True)
        epGuide = epGuide[u_indices]
        epTrue = epTrue[u_indices]
        iL = iL[u_indices]
        bL = bL[u_indices]
    
    return bL, iL, phiList, epGuide, epTrue, True

In [11]:
end_points = out_ep[1][79]
length_mod=1
dist_cut=6
phiQueryNum=10
neighbors = 10
randMult=10
maxPhi_cut=20
verbose= True

In [12]:
end_points.shape

(28, 3)

In [14]:
bL, iL, phiList, iL, iL, suc= add_loops(end_points, neighbors=10, length_mod=3, dist_cut=6, phiQueryNum=10, randMult=0, maxPhi_cut=20,
              uniquePhi=True, verbose=True)

first helix: #7
first loop: #203
second helix: #77
hnum 1
3 helix: reduced to #20
3 helix: #20
hnum 2
4 helix: reduced to #20
4 helix: #20
hnum 3
5 helix: #14
hnum 4
rn: 1
4 helix: reduced to #120
4 helix: #120
hnum 3
5 helix: reduced to #120
5 helix: #120
hnum 4
6 helix: reduced to #120
6 helix: #120
hnum 5
7 helix: reduced to #120
7 helix: #120
hnum 6
8 helix: reduced to #120
8 helix: #120
hnum 7
9 helix: reduced to #120
9 helix: #120
hnum 8
10 helix: reduced to #120
10 helix: #120
hnum 9
11 helix: #51
hnum 10
12 helix: reduced to #120
12 helix: #120
hnum 11
13 helix: reduced to #120
13 helix: #120
hnum 12
rn: 2
12 helix: reduced to #220
12 helix: #220
hnum 11
13 helix: reduced to #220
13 helix: #220
hnum 12
14 helix: #1


In [17]:
outDirec= 'output/'
x='test'
for num,prot in enumerate(bL):
    nu.dump_npdb(prot,f'{outDirec}build{x}_{num}.pdb')

In [13]:
def add_loops(end_points, neighbors=10, length_mod=1, dist_cut=6, phiQueryNum=10, randMult=10, maxPhi_cut=20,
              uniquePhi=True, verbose=False, maxReruns = 4):
    """Chains first_helix, first_loop, second helix and the iteratively loop_then_helix to add all loops to endpoints."""
    
    epIn = end_points.copy()
    size = int(len(epIn)/2) #number of helixes represented by helical endpoints
    
   
    bL, iL, epGuide = le.first_helix(epIn,length_mod=length_mod)
    if verbose:
        print(f'first helix: #{iL.shape[0]}')
    
    if iL.shape[0] == 0:
        if verbose:
            print('fail')
        return bL, iL, phiList, iL, iL, False
    
    bL, iL, epGuide, epTrue, phiList, loopFeature, xform_True = le.first_loop(bL, iL, epGuide, 
                                                                neighbors=neighbors, phiQueryNum=phiQueryNum,
                                                                           randMult=randMult, distCut=dist_cut)
    if verbose:
        print(f'first loop: #{iL.shape[0]}')
    if iL.shape[0] == 0:
        if verbose:
            print('fail')
        return bL, iL, phiList, iL, iL, False
    
    hnum=1
    bL, iL, epGuide, epTrue, phiList = le.second_helix(bL, iL, epGuide, epTrue, phiList, hnum, loopFeature, 
                                                   xform_True, length_mod=length_mod, distCut=dist_cut)
    if verbose:
        print(f'second helix: #{iL.shape[0]}')
    
    if iL.shape[0] == 0:
        if verbose:
            print('fail')
        return bL, iL, phiList, iL, iL, False
    
    repeatNumber = 0 
    hnum = 0
    
    bL_s = []
    iL_s = []
    epGuide_s = []
    epTrue_s = []
    phiList_s =  []
    for x in range(0,(size-1)):
        bL_s.append([])
        iL_s.append([])
        epGuide_s.append([])
        epTrue_s.append([])
        phiList_s.append([])
        
    
    while hnum<(size-2):
        hnum += 1
        print('hnum',hnum)
        bL, iL, epGuide, epTrue, phiList = le.next_loop_helix(bL, iL, epGuide, epTrue, phiList, hnum, 
                neighbors=neighbors, phiQueryNum=phiQueryNum, randMult=randMult, distCut=dist_cut,length_mod=length_mod)
       
        bL_s[hnum] = copy.deepcopy(bL)
        iL_s[hnum] = iL.copy()
        epGuide_s[hnum] = epGuide.copy()
        epTrue_s[hnum] = epTrue.copy()
        phiList_s[hnum] = phiList.copy()
        
        if iL.shape[0] == 0:
            
            hnum = hnum - 2
            
            bL = bL_s[hnum] 
            iL = iL_s[hnum] 
            epGuide = epGuide_s[hnum] 
            epTrue = epTrue_s[hnum] 
            phiList = phiList_s[hnum] 
            
            neighbors += 5
            maxPhi_cut += 100
            
            repeatNumber += 1
            print('rn:', repeatNumber)
                
            if repeatNumber == maxReruns:
                if verbose:
                    print('fail2')
                return bL, iL, phiList, iL, iL, False
            
        if iL.shape[0] > maxPhi_cut:
            indexer = le.random_reduce(bL,num_to_keep = maxPhi_cut)
            bL = bL[indexer]
            iL = iL[indexer] 
            epGuide = epGuide[indexer]
            epTrue =  epTrue[indexer]
            phiList = phiList[indexer]
            if verbose:
                print(f'{hnum+2} helix: reduced to #{maxPhi_cut}')
        if verbose:
            print(f'{hnum+2} helix: #{iL.shape[0]}')
            
         
    #only return proteins with a unique set of loops queried from different phi bins
    if uniquePhi:
        phiList, u_indices = np.unique(phiList,axis=0,return_index=True)
        epGuide = epGuide[u_indices]
        epTrue = epTrue[u_indices]
        iL = iL[u_indices]
        bL = bL[u_indices]
    
    return bL, iL, phiList, epGuide, epTrue, True

In [ ]:
def generate_parabola(start, stop, h=[20,20,20], k=[20,30,40], num_points = 100):
    #create a set of parabolas past through ([0,0]) start of helices to vertex ([[10,20],[10,30],[10,40]])
    # (x-h)^2 = -4(a)(y-k)
    # Solve for 'a' at x=0, y=0
    # (0-h)^2 = -4(a)(0-k)
    #      a  =  h^2/4k
    h = np.array(h).reshape((-1,1))
    k = np.array(k).reshape((-1,1))
    a = np.square(h)/(4*k)
    
    #trace x from 0 to 10
    # -4ay +4ak = (x-h)^2
    # -4ay = (x-h)^2 - 4ak
    #    y = ( (x-h)^2 - 4ak  ) / (-4a)
    #    y = (4ak - (x-h)^2)
    x = np.repeat(np.expand_dims(np.linspace(start,stop,num=100), axis=0), h.shape[0],axis=0)
    z = np.divide(4*a*k -np.square(x-h), (4*a))
   
    x=np.expand_dims(x,axis=2)
    z=np.expand_dims(z,axis=2)
    y = np.zeros_like(x)
    gen_para =  np.concatenate((x,y,z),axis = 2)
    
    return gen_para

In [29]:
def generate_pos_circle(start, stop, h=30, k=0, r=30, num_points= 100):
    """Generate guide points originating a the origin for a circle, positive values only. """
    #(x-h)^2 + (y-k)^2 = r^(2)
    # y = sqrt(r^2-(x-h^2))+k
    x = np.linspace(start,stop,num=100)
    z = np.sqrt(np.square(r)-np.square(x-h))+k
    y = np.zeros_like(x)
    return x,y,z

In [34]:
x,y,z = generate_pos_circle(0,30,h=30,k=0,r=30)

In [44]:
circ = np.concatenate((x.reshape((-1,1)),y.reshape((-1,1)),z.reshape((-1,1))),axis=1)